# Collecting and analysing median houdehold income and population Data on the United States

### This notebook should be ran first

In [9]:
# Run the following command in terminal pip install -r requirements.txt

Importing packages

In [54]:
import requests
import pandas as pd
import streamlit as st
import altair as alt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### creating data frame from US census data on populations of states in 2019

In [55]:
jsonlist = requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=state:*").json()
us_states = []
us_state_populations = []

for list in jsonlist:
    us_states.append(list[0])
    us_state_populations.append(list[1])
us_state_population_data = {"State":us_states,"Population":us_state_populations}
us_population_df = pd.DataFrame(us_state_population_data)
us_population_df = us_population_df.iloc[1:]
us_population_df["Population"] = pd.to_numeric(us_population_df["Population"])
us_population_df.head()





,State,Population
1,Mississippi,2976149
2,Missouri,6137428
3,Montana,1068778
4,Nebraska,1934408
5,Nevada,3080156


#### creating bar chart from data and experimenting with streamlit

In [56]:
chart = alt.Chart(us_population_df).mark_area().encode(
    x="State",
    y="Population"
)
st.title('Population Chart')
st.altair_chart(
    chart, use_container_width=True)


/Users/maddonk/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


DeltaGenerator()

Vega-altair uses its own id system that I could not find anywhere else so I used one of their datasets to retrieve their ids

In [57]:
from vega_datasets import data
# values in vega dataset stored alphabetically so dataframe was sorted alphabetically
us_population_df=us_population_df.sort_values(by="State")
us_population_df=us_population_df.reset_index(drop=True)

altair_state_ids = data.population_engineers_hurricanes()

us_population_df["id"]=altair_state_ids['id']
us_population_df.head()


,State,Population,id
0,Alabama,4903185,1
1,Alaska,731545,2
2,Arizona,7278717,4
3,Arkansas,3017804,5
4,California,39512223,6


In [58]:
from vega_datasets import data

states_geo = alt.topo_feature(data.us_10m.url, 'states')


alt.Chart(states_geo).mark_geoshape().encode(
    color='Population:Q',
    tooltip=['State:N', 'Population:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(us_population_df, 'id', ['Population','State'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)


alt.Chart(...)

Some data was not as easily available from the us census api so we decided to scrape their online data tables instead

In order to do this we needed the state codes as this is what the website used in their url to indicate different states

In [59]:
state_codes = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
state_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California",
    "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas",
    "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
    "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana",
    "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma",
    "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]

Here each state on us census 'quickfacts' is cycled through to collect the average household median income between 2017 and 2021 (2021 dollars)

In [60]:
from scrapy import Selector
state_median_incomes =[]
for state in state_codes:
   
    quickfacts_url= 'https://www.census.gov/quickfacts/fact/table/'+state+'/INC110221#INC110221'
   
    response = requests.get(quickfacts_url)
    sel= Selector(text=response.text)
    state_median_incomes.append(sel.css('td::text').getall()[3])

state_household_income_df = pd.DataFrame({"State":state_names,"Median Household income":state_median_incomes})
state_household_income_df.head()

,State,Median Household income
0,Alabama,"$54,943"
1,Alaska,"$80,287"
2,Arizona,"$65,913"
3,Arkansas,"$52,123"
4,California,"$84,097"


In [67]:
merged_population_income = pd.merge(us_population_df,state_household_income_df)
merged_population_income.head()

,State,Population,id,Median Household income
0,Alabama,4903185,1,"$54,943"
1,Alaska,731545,2,"$80,287"
2,Arizona,7278717,4,"$65,913"
3,Arkansas,3017804,5,"$52,123"
4,California,39512223,6,"$84,097"


We decided it would be better to use population density to compare against median household income 

here we are extracting the area of us states by scraping a wikipedia page

In [84]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area'
htmlresponse = pd.read_html(wiki_url,header=0)

In [87]:
state_area_df = htmlresponse[0]
state_area_df = state_area_df[['State / territory','Land mi2']]
state_area_df = state_area_df.rename(columns={'State / territory': 'State'})
state_area_df.head()


,State,Land mi2
0,Alaska,570641
1,Texas,261232
2,California,155779
3,Montana,145546
4,New Mexico,121298


Merging the two dataframes

In [89]:
merged_population_income = pd.merge(merged_population_income,state_area_df)
merged_population_income.head()

,State,Population,id,Median Household income,Land mi2,Population/SqMi,Income_Integer
0,Alabama,4903185,1,"$54,943",50645,96.814789,54943
1,Alaska,731545,2,"$80,287",570641,1.281971,80287
2,Arizona,7278717,4,"$65,913",113594,64.076597,65913
3,Arkansas,3017804,5,"$52,123",52035,57.995657,52123
4,California,39512223,6,"$84,097",155779,253.642808,84097


creating population density colum   



In [90]:
merged_population_income["Population/SqMi"] = merged_population_income['Population'] / merged_population_income['Land mi2']
merged_population_income.head()

,State,Population,id,Median Household income,Land mi2,Population/SqMi,Income_Integer
0,Alabama,4903185,1,"$54,943",50645,96.814789,54943
1,Alaska,731545,2,"$80,287",570641,1.281971,80287
2,Arizona,7278717,4,"$65,913",113594,64.076597,65913
3,Arkansas,3017804,5,"$52,123",52035,57.995657,52123
4,California,39512223,6,"$84,097",155779,253.642808,84097


I knew new jersey was Us most populated states so I checked that my data reflected this before proceeding

In [91]:
from vega_datasets import data

states_geo = alt.topo_feature(data.us_10m.url, 'states')
merged_population_income["Population/SqMi"]=pd.to_numeric(merged_population_income["Population/SqMi"])

alt.Chart(states_geo).mark_geoshape().encode(
    color='Population/SqMi:Q',
    tooltip=['State:N', 'Population/SqMi:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(merged_population_income, 'id', ['Population/SqMi','State'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)



alt.Chart(...)

creating integer value for median income

In [92]:
merged_population_income['Income_Integer'] = merged_population_income['Median Household income'].replace('[\$,]', '', regex=True).astype(int)
merged_population_income['Population/SqMi'] = pd.to_numeric(merged_population_income['Population/SqMi'], errors='coerce')
scatter = alt.Chart(merged_population_income).mark_circle().encode(
    y= alt.Y('Income_Integer',title="Median Household Income"),
    x='Population/SqMi',
    tooltip=['State:N', 'Income_Integer:Q', 'Population/SqMi:Q']
)
regressionline = scatter.transform_regression('Income_Integer', 'Population/SqMi').mark_line()
combined_chart = (scatter + regressionline)
combined_chart.interactive()


alt.LayerChart(...)